In [1]:
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import metrics

In [ ]:
df = pd.read_csv("PATH\\merged_normalized.csv", encoding='utf-8-sig', index_col=[0])
oddsy1 = df.copy() #We will use it shortly
df = df.drop(df.columns[range(10)], axis=1) #So that we only have numerical variables
custom_map = {'Wygrali gospodarze':0,'Remis':1,'Wygrali goście':2}
df = df.dropna()
df['Kto_wygrał'] = df['Kto_wygrał'].map(custom_map)
#print(df.head(10))
#print(df['Kto_wygrał'].isnull().sum())
y = df['Kto_wygrał']
X = df.drop(['Kto_wygrał'], axis=1)

In [3]:
correlation_values = df.corr()['Kto_wygrał'].drop('Kto_wygrał') #Correlation between each variable and result doesn't seem to be very strong
correlation_values

Punkty_Gospodarze          -0.073269
Zwycięstwa_Gospodarze      -0.074656
Remisy_Gospodarze           0.022528
Porażki_Gospodarze          0.067577
Gole_zdobyte_Gospodarze    -0.068445
Gole_stracone_Gospodarze    0.081916
Punkty_Goście               0.092149
Zwycięstwa_Goście           0.096079
Remisy_Goście              -0.041026
Porażki_Goście             -0.077287
Gole_zdobyte_Goście         0.066314
Gole_stracone_Goście       -0.081316
Name: Kto_wygrał, dtype: float64

In [4]:
oddsy1 = oddsy1[['Oddsy_gospodarze', 'Oddsy_remis', 'Oddsy_goście', 'Kto_wygrał']]
custom_map = {'Wygrali gospodarze':0,'Remis':1,'Wygrali goście':2}
oddsy1['Kto_wygrał'] = oddsy1['Kto_wygrał'].map(custom_map)
oddsy1 = oddsy1.dropna()
oddsy1.head()

,Oddsy_gospodarze,Oddsy_remis,Oddsy_goście,Kto_wygrał
0,1.94,3.81,3.67,1.0
1,2.08,3.80,3.33,1.0
2,1.36,5.21,8.39,0.0
3,1.96,3.95,3.49,2.0
4,1.38,5.37,7.25,1.0


##### ExpectedReturnNaive algorithm picks scenario with the highest probability, and returns what we would get back by placing bets according to this strategy

In [22]:
def ExpectedReturnNaive(oddsy, X, y, i, j, t, wyg, acc):
    data = oddsy.copy()
    X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=t, random_state=j)
    meczTree = DecisionTreeClassifier(criterion="log_loss", max_depth = i)
    meczTree.fit(X_trainset,y_trainset)
    tree_predictions = meczTree.predict(X_testset)
    indices = list(X_testset.index.astype(int))

    dane_tp = data.loc[indices] #dane_tp = dane_tree_predictions; assign testset to bookmaker odds
    dane_tp['Predicted_Winner'] = pd.Series(tree_predictions)
    dane_tp = dane_tp.dropna()
    dane_tp['Predicted_Winner'] = dane_tp['Predicted_Winner'].astype(int)
    dane_tp['Wygrana'] = 0.0
    dane_tp.loc[dane_tp.Kto_wygrał == dane_tp.Predicted_Winner,'Wygrana'] = 1
    dane_tp.loc[dane_tp.Kto_wygrał == 0,'Wygrana'] = dane_tp.Oddsy_gospodarze * 100 * dane_tp.Wygrana
    dane_tp.loc[dane_tp.Kto_wygrał == 1,'Wygrana'] = dane_tp.Oddsy_remis * 100 * dane_tp.Wygrana
    dane_tp.loc[dane_tp.Kto_wygrał == 2,'Wygrana'] = dane_tp.Oddsy_goście * 100 * dane_tp.Wygrana
    wyg = wyg + (dane_tp['Wygrana'].sum())/len(dane_tp['Wygrana'])
    acc = acc + 100*(round(metrics.accuracy_score(y_testset, tree_predictions),4))
    return wyg, acc, t

##### ExpectedReturnAdvanced algorithm calculates Expected Value of each bet (i.e. probability of scenario A x bookmaker odds for scenario A), picks the one with highest EV, and returns what it'd yield back per 100zł placed, over the course of 2015/16 - 2024/25 seasons

In [23]:
def ExpectedReturnAdvanced(df, oddsy, X, y, i, j, t, wyg, acc):

    X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=t, random_state=j)
    indices = list(X_trainset.index)
    oddsy_X_trainset = oddsy.loc[indices]
    meczTree = DecisionTreeClassifier(criterion="log_loss", max_depth = i)
    meczTree.fit(X_trainset,y_trainset)

    tree_predictions = meczTree.predict_proba(X_testset)
    tree_predictions = pd.DataFrame(tree_predictions)
    indices2 = list(X_testset.index)
    oddsy_tree_predictions = oddsy.loc[indices2]
    oddsy_tree_predictions = oddsy_tree_predictions.reset_index(drop=True)
    tabelka = oddsy_tree_predictions.merge(tree_predictions, left_index=True, right_index=True)

    tabelka['Kto_wygrał'] = df['Kto_wygrał'] #It's a bit of a mass, but it works
    tabelka['Potencjalny_return'] = 0
    tabelka['Potencjalny_return'] = np.where(tabelka['Kto_wygrał'] == 0, tabelka['Oddsy_gospodarze']*100, tabelka['Potencjalny_return'])
    tabelka['Potencjalny_return'] = np.where(tabelka['Kto_wygrał'] == 1, tabelka['Oddsy_remis']*100, tabelka['Potencjalny_return'])
    tabelka['Potencjalny_return'] = np.where(tabelka['Kto_wygrał'] == 2, tabelka['Oddsy_goście']*100, tabelka['Potencjalny_return'])
    tabelka['Potencjalny_return'] = tabelka['Potencjalny_return'].astype(int)
    tabelka['Return_gospodarze'] = tabelka['Oddsy_gospodarze'] * tabelka[0]
    tabelka['Return_remis'] = tabelka['Oddsy_remis'] * tabelka[1]
    tabelka['Return_goście'] = tabelka['Oddsy_goście'] * tabelka[2]
    tabelka["Max(EV)"] = tabelka[['Return_gospodarze', 'Return_remis', 'Return_goście']].max(axis=1)
    tabelka['Max(EV)'] = np.where(tabelka['Max(EV)'] == tabelka['Return_gospodarze'], 0, tabelka['Max(EV)'])
    tabelka['Max(EV)'] = np.where(tabelka['Max(EV)'] == tabelka['Return_remis'], 1, tabelka['Max(EV)'])
    tabelka['Max(EV)'] = np.where(tabelka['Max(EV)'] == tabelka['Return_goście'], 2, tabelka['Max(EV)'])
    tabelka['Max(EV)'] = tabelka['Max(EV)'].astype(int)
    tabelka['Actual_return'] = np.where(tabelka['Max(EV)'] == tabelka['Kto_wygrał'], tabelka['Potencjalny_return'], 0)
    return_calkowity = tabelka['Actual_return'].sum()
    return_per_100 = return_calkowity / len(tabelka)
    accuracy_score = 100*(np.count_nonzero(tabelka['Actual_return']))/len(tabelka['Actual_return'])
    wyg = wyg + return_per_100
    acc = acc + accuracy_score
    #print(f"return_calkowity for max_depth={i}, random_state={j}: {return_calkowity}")
    #print(f"return_per_100 for max_depth={i}, random_state={j}: {return_per_100:.3f}")

    return wyg, acc, t

In [24]:
for i in range(1,20):
    wygrana1 = 0
    accuracy1 = 0
    wygrana2 = 0
    accuracy2 = 0
    for j in range(1, 51):
        wygrana1, accuracy1, t = ExpectedReturnNaive(oddsy1, X, y, i, j, 0.2, wygrana1, accuracy1)
        wygrana2, accuracy2, t = ExpectedReturnAdvanced(df, oddsy1, X, y, i, j, 0.2, wygrana2, accuracy2)
    print(f"Naive algorithm | Dla max_depth={i}: Average accuracy score = {(accuracy1/50):.3f}%, Average wygrana per 100zł: {(wygrana1/50):.3f}") #50 repetitions, therefore to get an average we divide by 50
    print(f"Advanced algorithm | Dla max_depth={i}: Average accuracy score = {(accuracy2/50):.3f}%, Average wygrana per 100zł: {(wygrana2/50):.3f}")
    print("\n")

Naive algorithm | Dla max_depth=1: Average accuracy score = 43.161%, Average wygrana per 100zł: 99.029
Advanced algorithm | Dla max_depth=1: Average accuracy score = 36.717%, Average wygrana per 100zł: 130.935


Naive algorithm | Dla max_depth=2: Average accuracy score = 43.692%, Average wygrana per 100zł: 97.577
Advanced algorithm | Dla max_depth=2: Average accuracy score = 36.277%, Average wygrana per 100zł: 128.043


Naive algorithm | Dla max_depth=3: Average accuracy score = 43.560%, Average wygrana per 100zł: 97.325
Advanced algorithm | Dla max_depth=3: Average accuracy score = 36.022%, Average wygrana per 100zł: 126.269


Naive algorithm | Dla max_depth=4: Average accuracy score = 43.276%, Average wygrana per 100zł: 97.507
Advanced algorithm | Dla max_depth=4: Average accuracy score = 35.516%, Average wygrana per 100zł: 123.965


Naive algorithm | Dla max_depth=5: Average accuracy score = 42.783%, Average wygrana per 100zł: 98.445
Advanced algorithm | Dla max_depth=5: Average acc